In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

import torch
from torch.utils.data import Dataset, SubsetRandomSampler, random_split
from gnn_pytorch.utils import normalize
from data.ZINC.zinc_dataset import ZINC

/home/ayank/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

logger = logging.getLogger(__name__)

In [18]:
train = ZINC(root_dir='./data/ZINC', split='train')
val = ZINC(root_dir='./data/ZINC', split='val')
# test = ZINC(root_dir='./data/ZINC', split='test')

In [19]:
x,adj,y = train[6]

In [17]:
from gnn_pytorch.models import GNN_Config, VanillaGCN
van_gcn_config = GNN_Config(n_layers=3, inp_dim=33)
vanilla_gcn = VanillaGCN(van_gcn_config)

05/24/2022 13:22:52 - INFO - gnn_pytorch.models -   number of parameters: 5.468450e+05


In [11]:
from gnn_pytorch.trainer import Trainer, TrainerConfig

tconf = TrainerConfig(max_epochs=1)
trainer = Trainer(model=vanilla_gcn, train_dataset=train, val_dataset=val, config=tconf)

In [12]:
vanilla_gcn

VanillaGCN(
  (gcn_stack): GCN_Stack(
    (0): GraphConvLayer(
      (weight): Linear(in_features=33, out_features=32, bias=True)
    )
    (1): GraphConvLayer(
      (weight): Linear(in_features=32, out_features=32, bias=True)
    )
    (2): GraphConvLayer(
      (weight): Linear(in_features=32, out_features=32, bias=True)
    )
  )
  (node_embd): Embedding(28, 33)
  (readout): ReadOut(
    (w1): Linear(in_features=32, out_features=512, bias=True)
    (w2): Linear(in_features=512, out_features=512, bias=True)
    (w3): Linear(in_features=512, out_features=512, bias=True)
    (w4): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [21]:
pred, loss = vanilla_gcn(x,adj,y)
print(pred, loss)

tensor([[-0.8335]], grad_fn=<AddmmBackward0>) tensor(0.0357, grad_fn=<MseLossBackward0>)
